In [1]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

C:\MAD4AG


In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.cluster import DBSCAN
import geopandas
import folium
from folium.plugins import HeatMapWithTime
import warnings

warnings.filterwarnings('ignore')


C:\Users\aglar\AppData\Local\anaconda3\envs\pycharm\lib\site-packages\geopandas\_compat.py:115: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [3]:
file_name = f'./dbs/big_geodata_se/stops.parquet'

#Read each batchs file
df = pd.read_parquet(file_name)

# DBSCAN to detect jumps


In [4]:
def add_individual_clusters(data):
    # represent points consistently as (lat, lon)
    coords = data[['lat', 'lng']].values

    # define the number of kilometers in one radian
    kms_per_radian = 6371.0088

    # define epsilon as 500 kilometers, converted to radians for use by haversine
    epsilon = 190 / kms_per_radian

    db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    data['cluster'] = cluster_labels + 1
    return data




In [5]:
tqdm.pandas()
df = df.groupby('uid').progress_apply(add_individual_clusters)

print('The number of people: ', len(df['uid'].unique()))
print('The number of people having jump: ', len(df['uid'][df['cluster']>1].unique()))

  0%|          | 0/322919 [00:00<?, ?it/s]

The number of people:  322919
The number of people having jump:  120303


In [6]:
df['uid'] = df['uid']+'-'+df['cluster'].astype(str)

df.drop(columns=['cluster'], inplace=True)

# DBSCAN to detect clusters

In [7]:
def add_individual_clusters(data):
    # represent points consistently as (lat, lon)
    coords = data[['lat', 'lng']].values

    # define the number of kilometers in one radian
    kms_per_radian = 6371.0088

    # define epsilon as 0.1 kilometers, converted to radians for use by haversine
    epsilon = 0.1 / kms_per_radian

    db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(coords))
    cluster_labels = db.labels_
    data['cluster'] = cluster_labels + 1
    return data


In [8]:
tqdm.pandas()
df = df.groupby('uid').progress_apply(add_individual_clusters)

  0%|          | 0/516101 [00:00<?, ?it/s]

In [9]:
# the clusters in one individual's different jumps

df[df['uid'] == '9b0b36c9-b1c0-444e-8708-e2751915a587-1']

,uid,localtime,lat,lng,loc,h_s,dur,holiday_s,weekday_s,seq,cluster
17673691,9b0b36c9-b1c0-444e-8708-e2751915a587-1,2019-07-25 04:38:16,67.849996,20.229495,2,4.633333,249.350000,1,1,1,1
17673692,9b0b36c9-b1c0-444e-8708-e2751915a587-1,2019-07-25 12:17:57,67.849996,20.229495,2,12.283333,25.866667,1,1,2,1
17673693,9b0b36c9-b1c0-444e-8708-e2751915a587-1,2019-07-25 13:56:39,67.850541,20.228620,7,13.933333,18.133333,1,1,3,1
17673694,9b0b36c9-b1c0-444e-8708-e2751915a587-1,2019-07-25 14:44:50,67.865758,20.215365,22,14.733333,18.616667,1,1,4,2
17673695,9b0b36c9-b1c0-444e-8708-e2751915a587-1,2019-07-25 15:39:01,67.849996,20.229495,2,15.650000,21.650000,1,1,5,1
17673696,9b0b36c9-b1c0-444e-8708-e2751915a587-1,2019-07-25 19:46:47,67.859770,20.238931,21,19.766667,15.516667,1,1,6,3
17673697,9b0b36c9-b1c0-444e-8708-e2751915a587-1,2019-07-25 20:02:41,67.860784,20.240550,3,20.033333,19.183333,1,1,7,4
17673698,9b0b36c9-b1c0-444e-8708-e2751915a587-1,2019-07-25 21:13:23,67.860784,20.240550,3,21.216667,26.033333,1,1,8,4
17673699,9b0b36c9-b1c0-444e-8708-e2751915a587-1,2019-07-25 22:23:47,67.860784,20.240550,3,22.383333,76.083333,1,1,9,4
17673700,9b0b36c9-b1c0-444e-8708-e2751915a587-1,2019-07-25 23:51:49,67.860784,20.240550,3,23.850000,56.316667,1,1,10,4


In [10]:
df[df['uid'] == '9b0b36c9-b1c0-444e-8708-e2751915a587-2']

,uid,localtime,lat,lng,loc,h_s,dur,holiday_s,weekday_s,seq,cluster
17673716,9b0b36c9-b1c0-444e-8708-e2751915a587-2,2019-07-29 12:02:07,59.333924,18.056550,14,12.033333,29.850000,1,1,26,1
17673717,9b0b36c9-b1c0-444e-8708-e2751915a587-2,2019-07-29 12:57:14,59.331117,18.053898,5,12.950000,17.150000,1,1,27,2
17673718,9b0b36c9-b1c0-444e-8708-e2751915a587-2,2019-07-29 13:23:07,59.331117,18.053898,5,13.383333,26.083333,1,1,28,2
17673719,9b0b36c9-b1c0-444e-8708-e2751915a587-2,2019-07-29 14:11:37,59.332230,18.057570,13,14.183333,34.650000,1,1,29,3
17673720,9b0b36c9-b1c0-444e-8708-e2751915a587-2,2019-07-29 16:02:21,59.235395,18.231443,1,16.033333,270.516667,1,1,30,4
17673721,9b0b36c9-b1c0-444e-8708-e2751915a587-2,2019-07-30 00:33:17,59.235395,18.231443,1,0.550000,43.066667,1,1,31,4
17673722,9b0b36c9-b1c0-444e-8708-e2751915a587-2,2019-07-30 03:04:37,59.235395,18.231443,1,3.066667,433.950000,1,1,32,4
17673723,9b0b36c9-b1c0-444e-8708-e2751915a587-2,2019-07-30 13:02:45,59.331427,18.056124,4,13.033333,39.966667,1,1,33,5
17673724,9b0b36c9-b1c0-444e-8708-e2751915a587-2,2019-07-30 13:56:21,59.330968,18.056356,12,13.933333,33.350000,1,1,34,5
17673725,9b0b36c9-b1c0-444e-8708-e2751915a587-2,2019-07-30 15:31:22,59.235395,18.231443,1,15.516667,184.866667,1,1,35,4


## Drop individuals with only one cluster

In [11]:
ppl_with_one_cluster = df[['uid','cluster']].drop_duplicates(['uid','cluster'])

ppl_with_one_cluster = ppl_with_one_cluster.groupby(['uid']).size().reset_index(name='counts')
ppl_with_clusters = ppl_with_one_cluster['uid'][ppl_with_one_cluster['counts']>1].to_list()

df = df[df['uid'].isin(ppl_with_clusters)]

## Intersect MAD data and DeSO data

In [12]:
gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.lng, df.lat), crs="EPSG:4326")

In [13]:
DeSO = geopandas.read_file(f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/deso_statistik_shp/Bef_Kon_region.shp')

print(DeSO.crs)

DeSO.to_crs(4326, inplace=True)
print(DeSO.crs)

PROJCS["SWEREF99 TM",GEOGCS["SWEREF99",DATUM["SWEREF99",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6619"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",15],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","3006"]]
epsg:4326


In [14]:
gdf = gdf.sjoin(DeSO[['Deso','geometry']], how="left")

gdf.drop("index_right", axis=1, inplace=True)
#out of sweden
print('Stops outside Sweden: ', len(gdf[gdf['Deso'].isna()]))

gdf = gdf.dropna(subset=['Deso'])

df = pd.DataFrame(gdf.drop(columns='geometry'))


Stops outside Sweden:  81314


## Calculate mean of clusters and snap to the closest building

In [15]:

df_cluster = df.groupby(['uid','cluster'])[['lat','lng']].mean().reset_index()


## Snap the cluster centre to the closest building.

In [10]:
building_point = geopandas.read_file(f'C:/Synthetic_population_new/caglar/synthetic_sweden/input/Geodatabase.gdb', layer= 'Buildings_SpatJ_SU_1km_DESO_poi_wgs84')

building_point = pd.DataFrame(building_point[['TARGET_FID','ANDAMAL_1',
       'Deso', 'KNKOD',  'POINT_X_sweref99',
       'POINT_Y_sweref99', 'POINT_X', 'POINT_Y']])
building_point = building_point[building_point.ANDAMAL_1 != 699]
building_point.rename(columns={'POINT_Y': 'lat', 'POINT_X': 'lng'}, inplace=True)

In [16]:
#HERE
# df_cluster.to_parquet(f'./dbs/intermediate/df_cluster.parquet')
# building_point.to_parquet(f'./dbs/intermediate/building_point.parquet')


In [16]:
# df_cluster = pd.read_parquet(f'./dbs/intermediate/df_cluster.parquet')
# building_point = pd.read_parquet(f'./dbs/intermediate/building_point.parquet')


In [17]:
from lib.nearest_point import nearest_neighbor

In [18]:
closest_build = nearest_neighbor(df_cluster, building_point, return_dist=True)


In [19]:
closest_build= closest_build[['TARGET_FID', 'ANDAMAL_1', 'Deso', 'KNKOD', 'POINT_X_sweref99','POINT_Y_sweref99', 'lat', 'lng', 'distance']]

closest_build.rename(columns={'lat':'building_lat', 'lng':'building_lng'}, inplace=True)

df_cluster = df_cluster.join(closest_build)


In [20]:
df_cluster= df_cluster[['uid', 'cluster', 'lat', 'lng', 'TARGET_FID',
       'ANDAMAL_1', 'Deso', 'KNKOD', 'building_lng', 'building_lat', 'distance']]

df_cluster.rename(columns={'lat':'cluster_lat', 'lng':'cluster_lng'}, inplace=True)



### If the distance between the closest building and the cluster centre is more than 200m, use the coordinates of the cluster.

In [21]:
#HERE

df_cluster.cluster_lat = np.where(df_cluster['distance'] <= 0.2 , df_cluster.building_lat, df_cluster.cluster_lat)

df_cluster.cluster_lng = np.where(df_cluster['distance'] <= 0.2 , df_cluster.building_lng, df_cluster.cluster_lng)

df_cluster.ANDAMAL_1 = np.where(df_cluster['distance'] >= 0.2 , 0, df_cluster.ANDAMAL_1)

In [22]:
df_cluster.ANDAMAL_1.value_counts(normalize=True)

130    0.236211
133    0.215806
499    0.166975
0      0.095258
399    0.060074
799    0.036645
299    0.032780
319    0.022525
304    0.016702
131    0.014518
132    0.013796
240    0.009964
135    0.007733
318    0.007595
313    0.007160
247    0.007098
317    0.006719
310    0.005377
253    0.005355
320    0.004167
599    0.002749
199    0.002727
307    0.002502
301    0.002101
311    0.001898
242    0.001859
246    0.001820
249    0.001647
315    0.001303
303    0.001099
321    0.000979
314    0.000972
309    0.000967
324    0.000896
302    0.000830
243    0.000788
306    0.000553
248    0.000430
252    0.000266
316    0.000233
322    0.000200
308    0.000195
305    0.000147
312    0.000135
251    0.000122
250    0.000091
245    0.000024
241    0.000008
244    0.000003
Name: ANDAMAL_1, dtype: float64

In [23]:

df = pd.merge(df, df_cluster[['uid', 'cluster', 'cluster_lat', 'cluster_lng', 'building_lat','building_lng', 'ANDAMAL_1', 'distance']], on=['uid', 'cluster'], how='left' )

In [25]:
#HERE
df.to_parquet(f'./dbs/intermediate/stops_1_new.parquet')
